## Deliverable 2. Create a Customer Travel Destinations Map.

In [56]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [60]:
# 1. Import the WeatherPy_database.csv file. 
cities_data_df = pd.read_csv("../../Weather_Database/WeatherPy_Database.csv")
# cities_data_df.head()

In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# write two input statements that prompt the user to enter their minimum and maximum
# temperature criteria for their vacation
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = cities_data_df.loc[(cities_data_df["Max Temp"] <= max_temp) & \
                                       (cities_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Emerald,AU,-23.5333,148.1667,76.33,73,0,8.50,clear sky
13,13,Bairiki,KI,1.3292,172.9752,84.20,74,40,4.61,scattered clouds
17,17,Samarai,PG,-10.6167,150.6667,81.66,76,20,2.17,few clouds
19,19,Semporna,MY,4.4818,118.6112,77.20,88,99,7.43,overcast clouds
27,27,Muisne,EC,0.6000,-80.0333,77.11,87,100,11.21,overcast clouds
29,29,Atuona,PF,-9.8000,-139.0333,78.37,75,100,17.00,overcast clouds
31,31,Bengkulu,ID,-3.8004,102.2655,76.41,89,65,4.50,broken clouds
35,35,Vaini,TO,-21.2000,-175.2000,84.36,94,75,8.05,light rain
44,44,Souillac,MU,-20.5167,57.5167,79.18,78,75,12.66,broken clouds
46,46,Kapaa,US,22.0752,-159.3190,75.18,77,75,5.99,moderate rain


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                220
City                   220
Country                220
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_data = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_data[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Emerald,AU,76.33,clear sky,-23.5333,148.1667,
13,Bairiki,KI,84.20,scattered clouds,1.3292,172.9752,
17,Samarai,PG,81.66,few clouds,-10.6167,150.6667,
19,Semporna,MY,77.20,overcast clouds,4.4818,118.6112,
27,Muisne,EC,77.11,overcast clouds,0.6000,-80.0333,
29,Atuona,PF,78.37,overcast clouds,-9.8000,-139.0333,
31,Bengkulu,ID,76.41,broken clouds,-3.8004,102.2655,
35,Vaini,TO,84.36,light rain,-21.2000,-175.2000,
44,Souillac,MU,79.18,broken clouds,-20.5167,57.5167,
46,Kapaa,US,75.18,moderate rain,22.0752,-159.3190,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for indx, row in hotel_df.iterrows() :
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]   
    #add lat and lng to the params to search for hotels with 5000 meters
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[indx, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [81]:
# 7. Drop the rows where there is no Hotel Name.

import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.isnull().sum()
clean_hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
# hotel_df['Hotel Name']
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [82]:
# 8a. Create the output File (CSV)
#save the DataFrame as a CSV in a new folder for that file.

output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [85]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
# hotel_info

In [86]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))